### Connect to Drive

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/Progetto AN2DL

### Import libraries

In [2]:
# Fix randomness and hide warnings
seed = 5

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd()+'/configs/'

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

import numpy as np
np.random.seed(seed)

import logging

import random
random.seed(seed)

In [3]:
# Import tensorflow
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)
print(tf.__version__)

2.15.0


In [4]:
# Import other libraries
import pandas as pd
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
plt.rc('font', size=16)
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler

### Functions

In [5]:
def inspect_data(data, periods, categories, rows):
  figs, axs = plt.subplots(len(rows), 1, figsize=(17,17))
  for i, row in enumerate(rows):
        axs[i].plot(data[row])
        axs[i].set_title("Number " +str(row)+ ", Category " +str(categories[row]))
        axs[i].set_xlim(periods[row])
  plt.show()

In [6]:
def build_sequences(data, lenghts, window=200, stride=20, telescope=9):
  assert window % stride == 0

  dataset = []
  labels = []
  for i in np.arange(data.shape[0]):
    check = lenghts[i][0] % window - telescope
    size = (window - check) % window
    if size > 0 and data.shape[1] - lenghts[i][0] - size < 0:
      padding = np.zeros(size - (data.shape[1] - lenghts[i][0]), dtype='float32')
      temp_data = np.concatenate((padding, data[i]))
    else:
      temp_data = data[i].copy()

    for idx in np.arange(temp_data.shape[0]-lenghts[i][0]-size, temp_data.shape[0]-window-telescope, stride):
      dataset.append(temp_data[idx:idx+window])
      labels.append(temp_data[idx+window:idx+window+telescope])

  dataset = np.array(dataset)
  labels = np.array(labels)

  return dataset, labels

### Training settings

In [7]:
# Set variables used for training
epochs        = 200
batch_size    = 32
window        = 200
stride        = 200
telescope     = 9

early_stopping = tfk.callbacks.EarlyStopping(
                  monitor='val_loss',
                  mode='min',
                  patience=10,
                  restore_best_weights=True)

lr_scheduling = tfk.callbacks.ReduceLROnPlateau(
                  monitor='val_loss',
                  mode='min',
                  patience=5,
                  factor=0.99,
                  min_lr=1e-5)

callbacks = [early_stopping, lr_scheduling]

In [8]:
def build_CONV_LSTM_model(input_shape, output_shape):
    # Ensure the input time steps are at least as many as the output time steps
    assert input_shape[0] >= output_shape[0], "For this exercise we want input time steps to be >= of output time steps"

    # Define the input layer with the specified shape
    input_layer = tfkl.Input(shape=input_shape, name='input_layer')

    # Add a Bidirectional LSTM layer with 64 units
    x = tfkl.Bidirectional(tfkl.LSTM(64, return_sequences=True, name='lstm'), name='bidirectional_lstm')(input_layer)

    # Add a 1D Convolution layer with 128 filters and a kernel size of 1
    x = tfkl.Conv1D(128, 1, padding='same', activation='relu', name='conv')(x)

    # Add a final Convolution layer to match the desired output shape
    output_layer = tfkl.Conv1D(output_shape[1], 1, padding='same', name='output_layer')(x)

    # Calculate the size to crop from the output to match the output shape
    crop_size = output_layer.shape[1] - output_shape[0]

    # Crop the output to the desired length
    output_layer = tfkl.Cropping1D((0, crop_size), name='cropping')(output_layer)

    output_layer = output_layer[:,:,0]

    # Construct the model by connecting input and output layers
    model = tf.keras.Model(inputs=input_layer, outputs=output_layer, name='CONV_LSTM_model')

    # Compile the model with Mean Squared Error loss and Adam optimizer
    model.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer=tf.keras.optimizers.Adam())

    return model

### Training

In [ ]:
# Load data
data_train = np.load('training_dataset/split_dataset/data_train.npy', allow_pickle=True)
periods_train = np.load('training_dataset/split_dataset/periods_train.npy', allow_pickle=True)
categories_train = np.load('training_dataset/split_dataset/category_train.npy', allow_pickle=True)
data_val = np.load('training_dataset/split_dataset/data_val.npy', allow_pickle=True)
periods_val = np.load('training_dataset/split_dataset/periods_val.npy', allow_pickle=True)
categories_val = np.load('training_dataset/split_dataset/category_val.npy', allow_pickle=True)

inspect_data(data_train, periods_train, categories_train, [100, 40000, 14000])

In [ ]:
X_train, y_train = build_sequences(
    data = data_train,
    lenghts = np.diff(periods_train),
    window = window,
    stride = stride,
    telescope = telescope
)

X_val, y_val = build_sequences(
    data = data_val,
    lenghts = np.diff(periods_val),
    window = window,
    stride = stride,
    telescope = telescope
)

input_shape   = (X_train.shape[1],1)
output_shape  = (y_train.shape[1],1)

print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"X_val: {X_val.shape}, y_train: {y_val.shape}")
print(f"Input shape: {input_shape}, output shape: {output_shape}")

In [ ]:
# Define model
model = build_CONV_LSTM_model(input_shape, output_shape)
model.summary()
tfk.utils.plot_model(model, expand_nested=True, show_shapes=True)

In [ ]:
# Train the model
history = model.fit(
    x = X_train,
    y = y_train,
    batch_size = batch_size,
    epochs = epochs,
    validation_data=(X_val, y_val),
    callbacks = callbacks
).history

In [13]:
# Save model
model.save('StartingModel')

In [ ]:
# Plot important informations
best_epoch = np.argmin(history['val_loss'])
plt.figure(figsize=(17,4))
plt.plot(history['loss'], label='Training loss', alpha=.8, color='#ff7f0e')
plt.plot(history['val_loss'], label='Validation loss', alpha=.9, color='#5a9aa5')
plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
plt.title('Mean Squared Error')
plt.legend()
plt.grid(alpha=.3)
plt.show()

plt.figure(figsize=(18,3))
plt.plot(history['lr'], label='Learning Rate', alpha=.8, color='#ff7f0e')
plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
plt.legend()
plt.grid(alpha=.3)
plt.show()